<a href="https://colab.research.google.com/github/yulbeom/portfolio/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# One-Hot Encoding
###레이블 인코딩을 하고 원-핫 인코딩까지 하는 이유???
####-> size에서오는 편향성을 없에기위해서
####단점: 차원이 늘어나므로 연산량이 늘어난다

In [ ]:
pip install konlpy ###  nlpy -> Natural Language Processing for Python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 7.3 MB/s 
     |████████████████████████████████| 453 kB 24.1 MB/s 


In [ ]:
from nltk.corpus import stopwords   ### nltk -> Natural Language ToolKit
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt  
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words_list = stopwords.words('english')
print(len(stop_words_list))
print(stop_words_list[:10])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [ ]:
# One-Hot vector
# pip install konlpy
from konlpy.tag import Okt 
okt = Okt()
tokens = okt.morphs("나는 자연어 처리를 배운다") # 토큰화
print(tokens)

['나', '는', '자연어', '처리', '를', '배운다']


In [ ]:
### One-Hot Encoding
word_to_index = {word:index for index, word in enumerate(tokens)}
print(word_to_index)
print(len(word_to_index))
def one_hot_encoding(word, word_to_index): 
  one_hot_vector = [0]*(len(word_to_index))
  index = word_to_index[word]
  one_hot_vector[index] = 1
  return one_hot_vector
one_hot_encoding("자연어", word_to_index)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}
6


[0, 0, 1, 0, 0, 0]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
text = "나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print('단어 집합 : ', tokenizer.word_index)
sub_text = "점심 먹으러 갈래 메뉴는 햄버거 최고야"
encoded = tokenizer.texts_to_sequences([sub_text])[0] # 레이블 인코딩 
print(encoded)
one_hot = to_categorical(encoded) # One-Hot Encoding -> 벡터로 변환, 값은 1 , 방향이 서로 다른 
print(one_hot) # 왜 열이 8개???

단어 집합 :  {'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}
[2, 5, 1, 6, 3, 7]
[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


### Padding
패딩(Padding)


In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
raw_text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."
sentences = sent_tokenize(raw_text)

In [ ]:
vocab = {}
preprocessed_sentences = []
stop_words = set(stopwords.words('english')) # stopwords 불용어
for sentence in sentences:
  # 단어 토큰화
  tokenized_sentence = word_tokenize(sentence)
  result=[]
  
  for word in tokenized_sentence:
    word = word.lower()
    if word not in stop_words: # word가 불용어가 아니면
      if len(word)>2:          # word 길이가 3이상이면
        result.append(word)    # result 리스트에 넣는다
        if word not in vocab:    # word가 vocab 딕셔너리안에 없다면
          vocab[word] = 0        # word : 0 으로 vocab 딕셔너리안에 입력
        vocab[word] += 1         
  preprocessed_sentences.append(result)
print(preprocessed_sentences)
print(vocab)
print(vocab['barber'])

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}
8


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences) 
encoded = tokenizer.texts_to_sequences(preprocessed_sentences)
print(encoded)
# 단어의 최대 길이 확인
max_len = max(len(item) for item in encoded)
print('최대 길이:', max_len) # 한 문장안에 7(최대)개의 토큰

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]
최대 길이: 7


In [ ]:
# 모든 문장의 길이를 max로 맞춥니다. 길이가 7보다 짧은 문장에는 숫자 0을
# padded_np = [i+[0]*(max_len-len(i)) for i in encoded]
# padded_np
for sentence in encoded:
    while len(sentence) < max_len:
        sentence.append(0)
padded_np = np.array(encoded)
print(padded_np)

[[ 1  5  0  0  0  0  0]
 [ 1  8  5  0  0  0  0]
 [ 1  3  5  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [ 2  4  3  2  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  2  0  0  0  0]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13  0  0  0]]


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoded1 = tokenizer.texts_to_sequences(preprocessed_sentences)
print(encoded1)
padded = pad_sequences(encoded1)
print(padded)
padded_b = pad_sequences(encoded1, padding='post')
print(padded_b)
(padded_np == padded_b).all()
padded_max = pad_sequences(encoded, padding='post', maxlen=5)
print(padded_max)
padded_max1 = pad_sequences(encoded, padding='post', truncating='post', maxlen=5) # truncating ???
print(padded_max1)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]
[[ 0  0  0  0  0  1  5]
 [ 0  0  0  0  1  8  5]
 [ 0  0  0  0  1  3  5]
 [ 0  0  0  0  0  9  2]
 [ 0  0  0  2  4  3  2]
 [ 0  0  0  0  0  3  2]
 [ 0  0  0  0  1  4  6]
 [ 0  0  0  0  1  4  6]
 [ 0  0  0  0  1  4  2]
 [ 7  7  3  2 10  1 11]
 [ 0  0  0  1 12  3 13]]
[[ 1  5  0  0  0  0  0]
 [ 1  8  5  0  0  0  0]
 [ 1  3  5  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [ 2  4  3  2  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  2  0  0  0  0]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13  0  0  0]]
[[ 0  0  0  0  0]
 [ 5  0  0  0  0]
 [ 5  0  0  0  0]
 [ 0  0  0  0  0]
 [ 3  2  0  0  0]
 [ 0  0  0  0  0]
 [ 6  0  0  0  0]
 [ 6  0  0  0  0]
 [ 2  0  0  0  0]
 [ 3  2 10  1 11]
 [ 3 13  0  0  0]]
[[ 1  5  0  0  0]
 [ 1  8  5  0  0]
 [ 1  3  5  0  0]
 [ 9  2  0  0  0]
 [ 2  4  3  2  0]
 [ 3  2  0  0  0]
 [ 1  4  6  0  0]
 [ 1  4  6

# Data set
#### Training : Test
#### Traning : Validation : Test

# 한국어 패키지
pip install git+https://github.com/haven-jeon/PyKoSpacing.git


In [ ]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-sv2a23t3
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-sv2a23t3
     - 570.3 MB 97.8 MB/s
     |████████████████████████████████| 5.6 MB 13.8 MB/s 
     |████████████████████████████████| 14.8 MB 49.9 MB/s 
     |████████████████████████████████| 132 kB 65.1 MB/s 
     |████████████████████████████████| 462 kB 44.6 MB/s 
     |████████████████████████████████| 1.3 MB 56.8 MB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268580 sha256=0f066f55f7163eca72a341ddc53a2ef52656fad3fb2866e7d4fbed47c3f28d23
  Stored in directory: /tmp/pip-ephem-wheel-cache-6ignzqv6/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=71847f5b0a4e661aec80743f5

In [ ]:
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(" ",'')
print(new_sent)

김철수는극중두인격의사나이이광수역을맡았다.철수는한국유일의태권도전승자를가리는결전의날을앞두고10년간함께훈련한사형인유연재(김광수분)를찾으러속세로내려온인물이다.


In [ ]:
import tensorflow as tf
from pykospacing import Spacing
# tf.autograph.experimental.do_not_convert
spacing = Spacing()
kospacing_sent = spacing(new_sent)
print(sent)
print(kospacing_sent)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.
김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.


In [ ]:
pip install git+https://github.com/ssut/py-hanspell.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-l472ee22
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-l472ee22
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=015500e5201e5bddf8c8eb5a4c08beb7985bda36a24ef69858c5f04da2046ddd
  Stored in directory: /tmp/pip-ephem-wheel-cache-rz3wwpao/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker
sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지"
spell_checker.check(sent).checked

'맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지'

In [ ]:
# new_sent 대입 후 pykospacing과 비교
print("pyhanspell : ", spell_checker.check(new_sent))

pyhanspell :  Checked(result=True, original='김철수는극중두인격의사나이이광수역을맡았다.철수는한국유일의태권도전승자를가리는결전의날을앞두고10년간함께훈련한사형인유연재(김광수분)를찾으러속세로내려온인물이다.', checked='김철수는 극 중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연제(김광수 분)를 찾으러 속세로 내려온 인물이다.', errors=1, words=OrderedDict([('김철수는', 4), ('극', 4), ('중', 4), ('두', 4), ('인격의', 4), ('사나이', 4), ('이광수', 4), ('역을', 4), ('맡았다.', 4), ('철수는', 4), ('한국', 4), ('유일의', 4), ('태권도', 4), ('전승자를', 4), ('가리는', 4), ('결전의', 4), ('날을', 4), ('앞두고', 4), ('10년간', 4), ('함께', 4), ('훈련한', 4), ('사형인', 4), ('유연제(김광수', 4), ('분)를', 4), ('찾으러', 4), ('속세로', 4), ('내려온', 4), ('인물이다.', 4)]), time=0.41045618057250977)


In [ ]:
from konlpy.tag import Okt
tokenizer = Okt()
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정'))

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정']


In [ ]:
pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 10.4 MB/s 


In [ ]:
# Cohesion Probability(응집 확률) & Branching Entropy 기반
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

('2016-10-20.txt', <http.client.HTTPMessage at 0x7f307477a290>)

In [ ]:
corpus = DoublespaceLineCorpus("2016-10-20.txt")
print(len(corpus))

30091


In [ ]:
i = 0
for d in corpus:
  if len(d) > 0:
    print(i+1, d)
    i += 1
  if i == 10:
    break

1 19  1990  52 1 22
2 오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서

In [ ]:
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 1.895 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [ ]:
from soynlp.tokenizer import LTokenizer
scores = {word : score.cohesion_forward for word, score in word_score_table.items()} # cp?? be??
I_tokenizer = LTokenizer(scores=scores)
I_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [ ]:
from soynlp.tokenizer import MaxScoreTokenizer
maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

In [ ]:
from soynlp.normalizer import *
print(emoticon_normalize('앜ㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠ', num_repeats=2))

아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠㅠ


In [ ]:
print(repeat_normalize('와하하하하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하핫', num_repeats=2))

와하하핫
와하하핫
와하하핫


#### 형태소 분석 입력 : '은경이는 사무실로 갔습니다.'
#### 형태소 분석 결과 : ['은', '경이', '는', '사무실', '로', '갔습니다', '.']


In [ ]:
pip install customized_konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 881 kB 6.7 MB/s 


In [ ]:
from ckonlpy.tag import Twitter
twitter = Twitter()
twitter.morphs('은경이는 사무실로 갔습니다.')

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['은', '경이', '는', '사무실', '로', '갔습니다', '.']

In [ ]:
twitter.add_dictionary('은경이', 'Noun')
twitter.morphs('은경이는 사무실로 갔습니다.')

['은경이', '는', '사무실', '로', '갔습니다', '.']